In [55]:
import pandas as pd
import numpy as np
import os
from nilm_pre_processing import NilmPreProcessing

# Aggregate each device data

In [56]:
# base_path = "/opt/nilm-shared-data/nilm_device_detection/other_dataset/iawe/labeled_segments"
# devices = ["air_conditioner_1", "air_conditioner_2", "clothes_iron", "fridges", "washing_machine", "wet_appliance"]

# for device in devices:
#     device_dfs = []
#     device_dir = f"{base_path}/{device}"
#     for files in os.listdir(device_dir):
#         if files.endswith(".xlsx"):
#             device_dfs.append(pd.read_excel(f"{device_dir}/{files}"))
#     aggregate_device_df = pd.concat(device_dfs, axis=0)
#     aggregate_device_df = aggregate_device_df[["Time", "unix_ts", "Irms", "Urms", "AvgPowerFactor", "P", "Q", "S", "Label"]]
#     aggregate_device_df.to_csv(f"{base_path}/aggregate/{device}_aggregate.csv", index=False)

# Getting working time for each devices

In [57]:
nilm_pp = NilmPreProcessing()

In [59]:
base_path = "/opt/nilm-shared-data/nilm_device_detection/other_dataset/iawe/labeled_segments/aggregate"
device_dfs = []
for files in os.listdir(base_path):
    if files.endswith(".csv"):
        device_df = pd.read_csv(f"{base_path}/{files}")
        print(device_df.columns)
        device_dfs.append(device_df)

Index(['Time', 'unix_ts', 'Irms', 'Urms', 'AvgPowerFactor', 'P', 'Q', 'S',
       'Label'],
      dtype='object')
Index(['Time', 'unix_ts', 'Irms', 'Urms', 'AvgPowerFactor', 'P', 'Q', 'S',
       'Label'],
      dtype='object')
Index(['Time', 'unix_ts', 'Irms', 'Urms', 'AvgPowerFactor', 'P', 'Q', 'S',
       'Label'],
      dtype='object')
Index(['Time', 'unix_ts', 'Irms', 'Urms', 'AvgPowerFactor', 'P', 'Q', 'S',
       'Label'],
      dtype='object')
Index(['Time', 'unix_ts', 'Irms', 'Urms', 'AvgPowerFactor', 'P', 'Q', 'S',
       'Label'],
      dtype='object')
Index(['Time', 'unix_ts', 'Irms', 'Urms', 'AvgPowerFactor', 'P', 'Q', 'S',
       'Label'],
      dtype='object')


In [60]:
all_devices_df = pd.concat(device_dfs, axis=0)
len(all_devices_df)

2505510

In [61]:
all_devices_df = all_devices_df.dropna()
len(all_devices_df)

2505510

In [62]:
nilm_pp.count_labels(all_devices_df["Label"].tolist())

{'ac1_s1': 27818,
 'ac1_s2': 13227,
 'ac1_s3': 233152,
 'ac2_s1': 112109,
 'ac2_s2': 13186,
 'ac2_s3': 181806,
 'clothes_iron': 1673,
 'fridges_s1': 1896976,
 'washing_machine_s1': 13479,
 'wet_appliance_s1': 12084}

In [63]:
all_devices_df = all_devices_df.drop_duplicates(subset=["Time", "Label"])
len(all_devices_df)

2505510

In [64]:
def combine_device_data(time_array, label_array):
    """
    Combines data from multiple devices that are running at the same time.

    Args:
        device_data (list of tuples): Each tuple contains two lists - time array and label array.

    Returns:
        tuple: Combined time array and combined label array.
    """
    combined_time = []
    combined_label = []

    # Create a dictionary to store time as keys and labels as values
    time_label_dict = {}

    # Iterate over each device's data
    for i in range(len(time_array)):
        time = time_array[i]
        label = label_array[i]

        # If the time is already in the dictionary, append the label
        if time in time_label_dict:
            time_label_dict[time].append(label)
        else:
            # Otherwise, create a new entry in the dictionary
            time_label_dict[time] = [label]

    # Combine the labels for each time
    for time, labels in time_label_dict.items():
        combined_time.append(time)
        combined_label.append(", ".join(labels))

    return combined_time, combined_label

In [67]:
# Combine the data
combined_time, combined_label = combine_device_data(all_devices_df["Time"].tolist(), all_devices_df["Label"].tolist())

combination_df = pd.DataFrame({
    "Time": combined_time,
    "Label": combined_label
})

In [68]:
print(len(combination_df))
combination_df = combination_df.sort_values(["Time"])
nilm_pp.count_labels(combination_df["Label"])

2068241


{'ac1_s1': 22262,
 'ac1_s1, ac2_s1': 119,
 'ac1_s1, ac2_s2': 131,
 'ac1_s1, ac2_s3': 83,
 'ac1_s2': 12919,
 'ac1_s2, ac2_s1': 4,
 'ac1_s3': 34287,
 'ac1_s3, ac2_s1': 13909,
 'ac1_s3, ac2_s2': 2656,
 'ac1_s3, ac2_s3': 21699,
 'ac2_s1': 20674,
 'ac2_s2': 2159,
 'ac2_s3': 30695,
 'clothes_iron': 350,
 'fridges_s1': 1580048,
 'fridges_s1, ac1_s1': 4103,
 'fridges_s1, ac1_s1, ac2_s1': 128,
 'fridges_s1, ac1_s1, ac2_s2': 201,
 'fridges_s1, ac1_s1, ac2_s3': 729,
 'fridges_s1, ac1_s2': 249,
 'fridges_s1, ac1_s2, ac2_s1': 4,
 'fridges_s1, ac1_s2, ac2_s2': 2,
 'fridges_s1, ac1_s2, ac2_s3': 19,
 'fridges_s1, ac1_s3': 80588,
 'fridges_s1, ac1_s3, ac2_s1': 23891,
 'fridges_s1, ac1_s3, ac2_s2': 3504,
 'fridges_s1, ac1_s3, ac2_s3': 52095,
 'fridges_s1, ac2_s1': 53178,
 'fridges_s1, ac2_s2': 4496,
 'fridges_s1, ac2_s3': 76216,
 'fridges_s1, clothes_iron': 1319,
 'fridges_s1, washing_machine_s1': 8848,
 'washing_machine_s1': 4592,
 'wet_appliance_s1': 4386,
 'wet_appliance_s1, ac1_s1': 49,
 'wet_applia

In [72]:
combination_df.head()

,Time,Label
2019207,2013-07-01 00:00:00+05:30,ac2_s1
2019208,2013-07-01 00:00:01+05:30,ac2_s1
2019209,2013-07-01 00:00:02+05:30,ac2_s1
2019210,2013-07-01 00:00:03+05:30,ac2_s1
2019211,2013-07-01 00:00:04+05:30,ac2_s1


# Matching with the mains data

## Mains 1

In [69]:
mains_1_df = pd.read_csv("/opt/nilm-shared-data/nilm_device_detection/other_dataset/iawe/transform_data/mains_data_1.csv")
mains_0_df = pd.read_csv("/opt/nilm-shared-data/nilm_device_detection/other_dataset/iawe/transform_data/mains_data_0.csv")

In [70]:
print(len(mains_1_df))
mains_1_df.head()

2091765


,Time,unix_ts,Irms,P,Q,S,AvgPowerFactor
0,2013-07-01 00:00:00+05:30,1.372617e+09,3.12894,677.111,77.1329,695.849,0.973072
1,2013-07-01 00:00:03+05:30,1.372617e+09,3.20957,694.346,81.6198,713.954,0.972536
2,2013-07-01 00:00:06+05:30,1.372617e+09,3.22031,693.801,79.1498,716.361,0.968507
3,2013-07-01 00:00:07+05:30,1.372617e+09,3.18518,693.388,79.1911,708.450,0.978739
4,2013-07-01 00:00:10+05:30,1.372617e+09,2.70798,587.476,34.7509,602.125,0.975671


In [71]:
print(len(mains_0_df))
mains_0_df.head()

2091769


,Time,unix_ts,Irms,P,Q,S,AvgPowerFactor
0,2013-07-01 00:00:00+05:30,1.372617e+09,0.128665,15.7126,6.000030e-18,28.6370,0.548682
1,2013-07-01 00:00:03+05:30,1.372617e+09,0.134082,16.3620,6.000030e-18,29.8492,0.548155
2,2013-07-01 00:00:06+05:30,1.372617e+09,0.134063,14.9257,6.000030e-18,29.8458,0.500094
3,2013-07-01 00:00:07+05:30,1.372617e+09,0.134050,14.9314,6.000030e-18,29.8389,0.500401
4,2013-07-01 00:00:10+05:30,1.372617e+09,0.134618,14.9448,6.000030e-18,29.9555,0.498900


In [73]:
merged1_df = pd.merge(combination_df, mains_1_df, on="Time", how="right")
merged0_df = pd.merge(combination_df, mains_0_df, on="Time", how="right")

In [74]:
print(len(merged1_df)) 
merged1_df.head()

2091765


,Time,Label,unix_ts,Irms,P,Q,S,AvgPowerFactor
0,2013-07-01 00:00:00+05:30,ac2_s1,1.372617e+09,3.12894,677.111,77.1329,695.849,0.973072
1,2013-07-01 00:00:03+05:30,ac2_s1,1.372617e+09,3.20957,694.346,81.6198,713.954,0.972536
2,2013-07-01 00:00:06+05:30,ac2_s1,1.372617e+09,3.22031,693.801,79.1498,716.361,0.968507
3,2013-07-01 00:00:07+05:30,ac2_s1,1.372617e+09,3.18518,693.388,79.1911,708.450,0.978739
4,2013-07-01 00:00:10+05:30,ac2_s1,1.372617e+09,2.70798,587.476,34.7509,602.125,0.975671


In [75]:
print(len(merged0_df)) 
merged0_df.head()

2091769


,Time,Label,unix_ts,Irms,P,Q,S,AvgPowerFactor
0,2013-07-01 00:00:00+05:30,ac2_s1,1.372617e+09,0.128665,15.7126,6.000030e-18,28.6370,0.548682
1,2013-07-01 00:00:03+05:30,ac2_s1,1.372617e+09,0.134082,16.3620,6.000030e-18,29.8492,0.548155
2,2013-07-01 00:00:06+05:30,ac2_s1,1.372617e+09,0.134063,14.9257,6.000030e-18,29.8458,0.500094
3,2013-07-01 00:00:07+05:30,ac2_s1,1.372617e+09,0.134050,14.9314,6.000030e-18,29.8389,0.500401
4,2013-07-01 00:00:10+05:30,ac2_s1,1.372617e+09,0.134618,14.9448,6.000030e-18,29.9555,0.498900


In [76]:
# Drop rows with NaN values in any column
filtered0_df = merged0_df.dropna()
filtered1_df = merged1_df.dropna()

In [77]:
print(len(filtered0_df)) 
print(filtered0_df.head())

print(len(filtered1_df)) 
print(filtered1_df.head())

1391619
                        Time   Label       unix_ts      Irms        P  \
0  2013-07-01 00:00:00+05:30  ac2_s1  1.372617e+09  0.128665  15.7126   
1  2013-07-01 00:00:03+05:30  ac2_s1  1.372617e+09  0.134082  16.3620   
2  2013-07-01 00:00:06+05:30  ac2_s1  1.372617e+09  0.134063  14.9257   
3  2013-07-01 00:00:07+05:30  ac2_s1  1.372617e+09  0.134050  14.9314   
4  2013-07-01 00:00:10+05:30  ac2_s1  1.372617e+09  0.134618  14.9448   

              Q        S  AvgPowerFactor  
0  6.000030e-18  28.6370        0.548682  
1  6.000030e-18  29.8492        0.548155  
2  6.000030e-18  29.8458        0.500094  
3  6.000030e-18  29.8389        0.500401  
4  6.000030e-18  29.9555        0.498900  
1391616
                        Time   Label       unix_ts     Irms        P        Q  \
0  2013-07-01 00:00:00+05:30  ac2_s1  1.372617e+09  3.12894  677.111  77.1329   
1  2013-07-01 00:00:03+05:30  ac2_s1  1.372617e+09  3.20957  694.346  81.6198   
2  2013-07-01 00:00:06+05:30  ac2_s1  1.3726

In [78]:
filtered0_df = filtered0_df[["unix_ts", "Time", "Irms", "P", "AvgPowerFactor", "Q", "S", "Label"]].sort_values("unix_ts")
filtered1_df = filtered1_df[["unix_ts", "Time", "Irms", "P", "AvgPowerFactor", "Q", "S", "Label"]].sort_values("unix_ts")

In [79]:
fridge_ac1_df = filtered0_df[filtered0_df["Label"] == "fridges_s1, ac1_s1"]
test_df = fridge_ac1_df.head()
test_df

,unix_ts,Time,Irms,P,AvgPowerFactor,Q,S,Label
53059,1.372690e+09,2013-07-01 20:15:07+05:30,9.15218,2025.22,0.981535,291.026,2063.32,"fridges_s1, ac1_s1"
53060,1.372690e+09,2013-07-01 20:15:08+05:30,9.15362,2024.92,0.981280,286.013,2063.55,"fridges_s1, ac1_s1"
53061,1.372690e+09,2013-07-01 20:15:10+05:30,9.20202,2025.04,0.976130,294.917,2074.56,"fridges_s1, ac1_s1"
53062,1.372690e+09,2013-07-01 20:15:11+05:30,9.20078,2024.89,0.976199,290.331,2074.26,"fridges_s1, ac1_s1"
53063,1.372690e+09,2013-07-01 20:15:12+05:30,9.19890,2024.60,0.976280,280.248,2073.79,"fridges_s1, ac1_s1"


In [82]:
ac_s1_all_df = pd.read_csv("/opt/nilm-shared-data/nilm_device_detection/other_dataset/iawe/labeled_segments/aggregate/air_conditioner_1_aggregate.csv")
ac_s1_df = ac_s1_all_df[ac_s1_all_df["Time"].isin(test_df["Time"].tolist())]
ac_s1_df

,Time,unix_ts,Irms,Urms,AvgPowerFactor,P,Q,S,Label
133599,2013-07-01 20:15:07+05:30,1372689907,1.974,224.06,0.907585,401.404,185.698,442.277,ac1_s1
133600,2013-07-01 20:15:08+05:30,1372689908,1.973,223.76,0.908223,401.064,184.800,441.592,ac1_s1
133602,2013-07-01 20:15:10+05:30,1372689910,1.945,223.44,0.906372,393.891,183.602,434.580,ac1_s1
133603,2013-07-01 20:15:11+05:30,1372689911,1.943,223.29,0.906741,393.433,182.970,433.898,ac1_s1
133604,2013-07-01 20:15:12+05:30,1372689912,1.945,223.16,0.907206,393.830,182.625,434.113,ac1_s1


In [84]:
fridge_s1_all_df = pd.read_csv("/opt/nilm-shared-data/nilm_device_detection/other_dataset/iawe/labeled_segments/aggregate/fridges_aggregate.csv")
fridge_s1_df = fridge_s1_all_df[fridge_s1_all_df["Time"].isin(test_df["Time"].tolist())]
fridge_s1_df

,Time,unix_ts,Irms,Urms,AvgPowerFactor,P,Q,S,Label
519841,2013-07-01 20:15:07+05:30,1372689907,0.453,225.23,0.854624,87.287,53.034,102.135,fridges_s1
519842,2013-07-01 20:15:08+05:30,1372689908,0.453,225.06,0.854691,87.193,52.961,102.017,fridges_s1
519844,2013-07-01 20:15:10+05:30,1372689910,0.453,224.64,0.855838,87.150,52.671,101.830,fridges_s1
519845,2013-07-01 20:15:11+05:30,1372689911,0.453,224.44,0.856300,87.042,52.500,101.649,fridges_s1
519846,2013-07-01 20:15:12+05:30,1372689912,0.453,224.32,0.855526,86.995,52.649,101.686,fridges_s1


In [85]:
fridge_ac1_df = filtered1_df[filtered1_df["Label"] == "fridges_s1, ac1_s1"]
test_df = fridge_ac1_df.head()
test_df

,unix_ts,Time,Irms,P,AvgPowerFactor,Q,S,Label
53059,1.372690e+09,2013-07-01 20:15:07+05:30,11.8241,2622.72,0.985570,150.769,2661.12,"fridges_s1, ac1_s1"
53060,1.372690e+09,2013-07-01 20:15:08+05:30,11.7389,2622.60,0.992721,150.857,2641.83,"fridges_s1, ac1_s1"
53061,1.372690e+09,2013-07-01 20:15:10+05:30,11.7976,2624.19,0.988340,148.505,2655.15,"fridges_s1, ac1_s1"
53062,1.372690e+09,2013-07-01 20:15:11+05:30,11.7986,2625.14,0.988645,150.491,2655.29,"fridges_s1, ac1_s1"
53063,1.372690e+09,2013-07-01 20:15:12+05:30,11.8014,2626.48,0.988960,140.354,2655.80,"fridges_s1, ac1_s1"


In [ ]:
ac_s1_all_df = pd.read_csv("/opt/nilm-shared-data/nilm_device_detection/other_dataset/iawe/labeled_segments/aggregate/air_conditioner_1_aggregate.csv")
ac_s1_df = ac_s1_all_df[ac_s1_all_df["Time"].isin(test_df["Time"].tolist())]
ac_s1_df

,Time,unix_ts,Irms,Urms,AvgPowerFactor,P,Q,S,Label
133599,2013-07-01 20:15:07+05:30,1372689907,1.974,224.06,0.907585,401.404,185.698,442.277,ac1_s1
133600,2013-07-01 20:15:08+05:30,1372689908,1.973,223.76,0.908223,401.064,184.800,441.592,ac1_s1
133602,2013-07-01 20:15:10+05:30,1372689910,1.945,223.44,0.906372,393.891,183.602,434.580,ac1_s1
133603,2013-07-01 20:15:11+05:30,1372689911,1.943,223.29,0.906741,393.433,182.970,433.898,ac1_s1
133604,2013-07-01 20:15:12+05:30,1372689912,1.945,223.16,0.907206,393.830,182.625,434.113,ac1_s1


In [ ]:
fridge_s1_all_df = pd.read_csv("/opt/nilm-shared-data/nilm_device_detection/other_dataset/iawe/labeled_segments/aggregate/fridges_aggregate.csv")
fridge_s1_df = fridge_s1_all_df[fridge_s1_all_df["Time"].isin(test_df["Time"].tolist())]
fridge_s1_df

,Time,unix_ts,Irms,Urms,AvgPowerFactor,P,Q,S,Label
519841,2013-07-01 20:15:07+05:30,1372689907,0.453,225.23,0.854624,87.287,53.034,102.135,fridges_s1
519842,2013-07-01 20:15:08+05:30,1372689908,0.453,225.06,0.854691,87.193,52.961,102.017,fridges_s1
519844,2013-07-01 20:15:10+05:30,1372689910,0.453,224.64,0.855838,87.150,52.671,101.830,fridges_s1
519845,2013-07-01 20:15:11+05:30,1372689911,0.453,224.44,0.856300,87.042,52.500,101.649,fridges_s1
519846,2013-07-01 20:15:12+05:30,1372689912,0.453,224.32,0.855526,86.995,52.649,101.686,fridges_s1
